# Time Series Forecasting Models
- In this task, we build a time series forecasting model to predict Tesla's (TSLA) future stock prices. 
- We explore and compare different modeling techniques, including ARIMA, SARIMA, and LSTM. 
- The goal is to develop a robust model that can accurately forecast stock prices and provide actionable insights for portfolio management.

In [1]:
import os, sys

notebook_dir = os.getcwd()

#add path
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))

In [2]:
from scripts.fetchPrepr_close_column import *
from scripts.sarima_model import *
from scripts.lstm_model import *

In [3]:
# import pandas as pd
# Fetch and preprocess data
tsla_data = fetch_data('TSLA', '2015-01-01', '2025-01-31')
train_data, test_data = preprocess_data(tsla_data)

# index = pd.date_range(start='2023-01-24', periods=len(data['Close']), freq='D')
# test_data = pd.DataFrame(test_data, index=index)

# Extract the 'Close' column as a Series
# test_data_series = test_data['Close']

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [4]:
print(train_data)
print(test_data)
# test_data.shape

Ticker            TSLA
Date                  
2015-01-02   14.620667
2015-01-05   14.006000
2015-01-06   14.085333
2015-01-07   14.063333
2015-01-08   14.041333
...                ...
2023-01-17  131.490005
2023-01-18  128.779999
2023-01-19  127.169998
2023-01-20  133.419998
2023-01-23  143.750000

[2028 rows x 1 columns]
Ticker            TSLA
Date                  
2023-01-24  143.889999
2023-01-25  144.429993
2023-01-26  160.270004
2023-01-27  177.899994
2023-01-30  166.660004
...                ...
2025-01-24  406.579987
2025-01-27  397.149994
2025-01-28  398.089996
2025-01-29  389.100006
2025-01-30  400.279999

[507 rows x 1 columns]


In [6]:
print(f"NaN values in test_data: {train_data.isnull().sum()}")
print(f"NaN values in test_data: {test_data.isnull().sum()}")
# print(f"NaN values in predictions: {pd.isnull(predictions).sum()}")

NaN values in test_data: Ticker
TSLA    0
dtype: int64
NaN values in test_data: Ticker
TSLA    0
dtype: int64


In [4]:
# SARIMA Model
print("Training SARIMA model...")
sarima_model = fit_sarima(train_data)

c:\Users\pc\Desktop\10_Academy\Week-11\kaim-week-11\.myvenv11\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\pc\Desktop\10_Academy\Week-11\kaim-week-11\.myvenv11\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Training SARIMA model...


In [5]:
# SARIMA Model
sarima_predictions, sarima_mae, sarima_rmse, sarima_mape = evaluate_sarima(sarima_model, test_data['TSLA'])
print(f"SARIMA Metrics - MAE: {sarima_mae}, RMSE: {sarima_rmse}, MAPE: {sarima_mape}")

test_data head:
Date
2023-01-24    143.889999
2023-01-25    144.429993
2023-01-26    160.270004
2023-01-27    177.899994
2023-01-30    166.660004
Name: TSLA, dtype: float64
predictions head:
2028    142.335427
2029    142.361755
2030    140.949504
2031    140.900303
2032    139.419443
Name: predicted_mean, dtype: float64
SARIMA Metrics - MAE: 91.82462746724023, RMSE: 112.59495522395002, MAPE: nan


c:\Users\pc\Desktop\10_Academy\Week-11\kaim-week-11\.myvenv11\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\pc\Desktop\10_Academy\Week-11\kaim-week-11\.myvenv11\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
c:\Users\pc\Desktop\10_Academy\Week-11\kaim-week-11\scripts\sarima_model.py:68: RuntimeWarning: '<' not supported between instances of 'int' and 'Timestamp', sort order is undefined for incomparable objects.
  mape = np.mean(np.abs((test_data - predictions) / test_data)) * 100
